In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-08 06:53:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.21MB/s    in 0.7s    

2022-03-08 06:53:56 (1.21 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://jk-data-bootcamp.s3.amazonaws.com/amazon_reviews_us_Home_Entertainment_v1_00.tsv"
spark.sparkContext.addFile(url)


In [5]:
reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Entertainment_v1_00.tsv"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy-MM-dd")
# Show DataFrame
reviews_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...|2015-08-31 00:00:00|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with

In [6]:
reviews_df.count()
print(f"There are total of {reviews_df.count()} reviews in the file.")

There are total of 705889 reviews in the file.


In [7]:
# Checking for any duplicate reviews based on the review ID
reviews_df.groupby(['review_id']).count().where('count>1').sort('count',ascending=False).show()
# no duplicates found under the review_id

+---------+-----+
|review_id|count|
+---------+-----+
+---------+-----+



In [8]:
reviews_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [9]:
print(reviews_df.count())
reviews_df = reviews_df.dropna()
print(reviews_df.count())

705889
705850


In [10]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [11]:
review_id_df = reviews_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RY01SAV7HZ8QO|     179886|B00NTI0CQ2|     667358431|2015-08-31 00:00:00|
|R1XX8SDGJ4MZ4L|   37293769|B00BUCLVZU|     621695622|2015-08-31 00:00:00|
|R149Q3B5L33NN5|    8332121|B00RBX9D5W|     143071132|2015-08-31 00:00:00|
|R2ZVD69Z6KPJ4O|   47054962|B00UJ3IULO|     567816707|2015-08-31 00:00:00|
|R1DIKG2G33ZLNP|   23413911|B0037UCTXG|     909557698|2015-08-31 00:00:00|
+--------------+-----------+----------+--------------+-------------------+
only showing top 5 rows



In [12]:
product = reviews_df.select(["product_id", "product_title"])
product_df = product.dropDuplicates()
print(product_df.count())
product_df.show(5)

44810
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00N9OT6RM|Upstar 19-Inch 72...|
|B007R9RUPU|Kinivo LS210 Port...|
|B00U9U9AAM|Samsung J6200, SB...|
|B00QCLTOQM|Panasonic TC32A40...|
|B007F9XJW0|Sony DVPFX780 7-I...|
+----------+--------------------+
only showing top 5 rows



In [13]:

customers_count = reviews_df.groupby(['customer_id']).count()
customer_col = customers_count.select("customer_id","count")
customer_df = customer_col.withColumnRenamed('count', 'customer_count')
customer_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10142992|             1|
|   16457323|             6|
|   11935383|             1|
|   46277736|             1|
|   13671072|             1|
+-----------+--------------+
only showing top 5 rows



In [14]:
vine_df = reviews_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RY01SAV7HZ8QO|          4|            0|          0|   N|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|
|R149Q3B5L33NN5|          5|            0|          0|   N|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



#Postgres Setup

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cuyhrvoj4eve.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user": "<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}


In [16]:
# Write DataFrame to review_id_table table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
# Write dataframe to products table in RDS

product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [18]:
# Write dataframe to customers table in RDS

customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [19]:
# Write dataframe to vine_table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# Analysis

In [27]:
# reading vine_table from AWS RDS database
df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
print(df.count())
df.show(5)

705850
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RY01SAV7HZ8QO|          4|            0|          0|   N|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|
|R149Q3B5L33NN5|          5|            0|          0|   N|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [28]:
# Filter dataframe to show total vine reviews
vine_df = df.filter(df["vine"] == "Y")
print(vine_df.count())
vine_df.show(5)

2106
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3DG19N0RJF1QY|          5|            0|          1|   Y|
|R3LLICBKRHLRJC|          4|           17|         22|   Y|
| RJRUL44DZXFXU|          4|            1|          1|   Y|
|R1XKEXPN5SJP3J|          2|           11|         15|   Y|
|R35S0Q8N0F6RNR|          4|            2|          2|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [29]:
vine_df.groupby("star_rating").count().sort("star_rating", ascending=False).show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          5|  854|
|          4|  761|
|          3|  328|
|          2|  118|
|          1|   45|
+-----------+-----+



In [30]:
vine_df.groupby("star_rating").sum().sort("star_rating", ascending=False).show()

+-----------+----------------+------------------+----------------+
|star_rating|sum(star_rating)|sum(helpful_votes)|sum(total_votes)|
+-----------+----------------+------------------+----------------+
|          5|            4270|              9961|           12057|
|          4|            3044|             10208|           11921|
|          3|             984|              3632|            4561|
|          2|             236|               835|            1190|
|          1|              45|               499|             731|
+-----------+----------------+------------------+----------------+



In [31]:
vine_vote_df = vine_df.groupby().sum()
vine_vote_df.select("sum(helpful_votes)","sum(total_votes)").show()

+------------------+----------------+
|sum(helpful_votes)|sum(total_votes)|
+------------------+----------------+
|             25135|           30460|
+------------------+----------------+



In [32]:
from pyspark.sql.functions import avg
vine_df.select(avg("star_rating")).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|4.073599240265907|
+-----------------+



In [34]:
# Filter dataframe to show non vine reviews
non_vine_df = df.filter(df["vine"] == "N")
print(non_vine_df.count())
non_vine_df.show(5)

703744
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RY01SAV7HZ8QO|          4|            0|          0|   N|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|
|R149Q3B5L33NN5|          5|            0|          0|   N|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [35]:
non_vine_df.groupby("star_rating").count().sort("star_rating", ascending=False).show()

+-----------+------+
|star_rating| count|
+-----------+------+
|          5|373107|
|          4|130746|
|          3| 56835|
|          2| 43648|
|          1| 99408|
+-----------+------+



In [36]:
non_vine_df.groupby("star_rating").sum().sort("star_rating", ascending=False).show()

+-----------+----------------+------------------+----------------+
|star_rating|sum(star_rating)|sum(helpful_votes)|sum(total_votes)|
+-----------+----------------+------------------+----------------+
|          5|         1865535|           1239409|         1458858|
|          4|          522984|            464299|          552310|
|          3|          170505|            227992|          311461|
|          2|           87296|            153863|          234145|
|          1|           99408|            438309|          729507|
+-----------+----------------+------------------+----------------+



In [37]:
non_vine_df.select(avg("star_rating")).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|3.901600582029829|
+-----------------+



In [38]:
df = non_vine_df.groupby().sum()
df.select("sum(helpful_votes)","sum(total_votes)").show()

+------------------+----------------+
|sum(helpful_votes)|sum(total_votes)|
+------------------+----------------+
|           2523872|         3286281|
+------------------+----------------+



## Conclusion

There is no significant 
Based on the below analysis, there is no significant evidience that shows that the vine reviews are trustworthy.
* Total number of vine reviews only makes up 0.2% of the overall review (2106 / 703744)
* star rating between the vine reviews vs non vine reviews averaged around in the similiar range - Average star rating for vine reviews (4.07) vs non vine reviews (3.90)
* helpful vote comparison : vine review 82.5% (25135/30460) vs non vine review 76.8% (2523872/3286281)